## 1. Install Dependencies

In [ ]:
!pip install -q transformers peft datasets bitsandbytes accelerate

## 2. Imports

In [ ]:
import json
import re
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
import torch

## 3. Load & Preprocess Chat Data

In [ ]:
# Example: Slack JSON export (replace 'slack_export.json' with your file)
# Slack/Teams exports usually look like:
# [
#   {"user": "Alice", "text": "Hello Bob", "ts": "1680000000.000000"},
#   {"user": "Bob", "text": "Hey Alice!", "ts": "1680000001.000000"}
# ]
#
# Upload your export file to Colab: Runtime → Files → Upload
CHAT_FILE = "/content/slack_export.json"  # change this

with open(CHAT_FILE, "r") as f:
    raw_data = json.load(f)

# Remove system messages and empty texts
cleaned = [
    msg for msg in raw_data
    if msg.get("text") and not msg["text"].startswith("<@") and "joined" not in msg["text"].lower()
]

# Convert into conversation turns (instruction → response)
pairs = []
for i in range(len(cleaned) - 1):
    current = cleaned[i]
    nxt = cleaned[i + 1]
    if current["user"] != nxt["user"]:  # only consecutive different speakers
        instruction = f"{current['user']}: {current['text']}\n{nxt['user']}:"
        output = nxt["text"]
        pairs.append({
            "instruction": "Respond in the style of our team chat",
            "input": instruction,
            "output": output
        })

print(f"Prepared {len(pairs)} conversation pairs.")

## 4. Create Dataset

In [ ]:
dataset = Dataset.from_list(pairs)

## 5. Tokenization Function

In [ ]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(example):
    prompt = f"{example['instruction']}\n{example['input']}"
    labels = example['output']
    text = f"{prompt}\n{labels}"
    tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_ds = dataset.map(tokenize_fn)

## 6. Load Model with QLoRA

In [ ]:
from peft import LoraConfig

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    #load_in_4bit=True,
    device_map="auto"
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

## 7. Training

In [ ]:
training_args = TrainingArguments(
    output_dir="tinyllama-slack",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds
)

trainer.train()

## 8. Save LoRA Adapters

In [ ]:
model.save_pretrained("tinyllama-slack-lora")
tokenizer.save_pretrained("tinyllama-slack-lora")
print("✅ Fine-tuning complete. Adapters saved to 'tinyllama-slack-lora'.")

## 9. Inference Example

In [ ]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16
)
ft_model = PeftModel.from_pretrained(base_model, "tinyllama-slack-lora")

prompt = "You are answering a question like a conversation.\n Dana: We need to update the docs. \nAlice:"
inputs = tokenizer(prompt, return_tensors="pt").to(ft_model.device)
outputs = ft_model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))